# 実行環境の作成

## Google Drive

Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Google Sheets

ライブラリのインストール

In [2]:
!pip install --upgrade -q gspread

Google Sheetsの認証

In [3]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Google Cloud Storage

下記コードでGCPに接続

In [4]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

## ESTAT_APP_ID

In [7]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [8]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# 統計カードのデータセット

## RESAS_API_KEY

In [14]:
RESAS_API_KEY = '	02JYzLqUOfTNVfiTWjLlR2g0YwEQPQ7caYxcnZaZ'

## 都道府県コードリスト

resas-APIから都道府県一覧を取得する

In [15]:
"""
resas-APIから都道府県一覧を取得する関数
"""

def get_resas_preflist():
    url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())
    return res['result']

estat-APIのリクエストパラメータ'cdArea'に都道府県コードをセットする

返却値はリスト

In [26]:
"""
estat-APIのパラメータ'cdArea'に都道府県コードをセットする関数
"""

def get_cdAreas_prefecture():

  pref_codes = [d.get('prefCode') for d in get_resas_preflist()]
  pref_codes = [str(c).zfill(5) for c in pref_codes]

  res = [{'cdArea':",".join(pref_codes)}]
  
  return res

In [27]:
cdArea = get_cdAreas_prefecture()
print(cdArea)

[{'cdArea': '00001,00002,00003,00004,00005,00006,00007,00008,00009,00010,00011,00012,00013,00014,00015,00016,00017,00018,00019,00020,00021,00022,00023,00024,00025,00026,00027,00028,00029,00030,00031,00032,00033,00034,00035,00036,00037,00038,00039,00040,00041,00042,00043,00044,00045,00046,00047'}]


## 市区町村コードリスト

resas-APIから市区町村一覧を取得する

In [16]:
"""
resas-APIから市区町村一覧を取得する関数
"""

def get_resas_citylist(prefCode=0,designatedCity='all'):
  
    # 都道府県'prefCode'を指定
    # 0の場合は全都道府県の市区町村
    if prefCode == 0:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities'
    else:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities?prefCode={}'.format(prefCode)
    
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())


    # 政令指定都市の指定　'join':政令指定都市統合　'split'：政令指定都市分割 'all'：全て
    if designatedCity == 'join':
      return list(filter(lambda x: x['bigCityFlag'] != '1', res['result']))
    elif designatedCity == 'split':
      return list(filter(lambda x: x['bigCityFlag'] != '2', res['result']))
    else:
      return res['result']

estat-APIのリクエストパラメータ'cdArea'に市区町村コードをセットする

返却値はリスト

In [17]:
"""
estat-APIのパラメータ'cdArea'に市区町村コードをセットする関数
"""

def get_cdAreas_city(prefCode=0):

  res=[]

  citylist = get_resas_citylist(prefCode=prefCode,designatedCity='all')
  city_codes = [d.get('cityCode') for d in citylist]

  # 市区町村が100以上ある場合
  if len(city_codes) >= 100 :
    for i in range(0, len(city_codes), 100):
      dic = {'cdArea':",".join(city_codes[i: i+100])}
      res.append(dic)

  # 市区町村が100未満の場合
  else:
    dic = {'cdArea':",".join(city_codes)}
    res.append(dic)

  return res

In [18]:
cdArea = get_cdAreas_city()
print(cdArea)
print('要素数：' + str(len(cdArea)))

[{'cdArea': '01100,01101,01102,01103,01104,01105,01106,01107,01108,01109,01110,01202,01203,01204,01205,01206,01207,01208,01209,01210,01211,01212,01213,01214,01215,01216,01217,01218,01219,01220,01221,01222,01223,01224,01225,01226,01227,01228,01229,01230,01231,01233,01234,01235,01236,01303,01304,01331,01332,01333,01334,01337,01343,01345,01346,01347,01361,01362,01363,01364,01367,01370,01371,01391,01392,01393,01394,01395,01396,01397,01398,01399,01400,01401,01402,01403,01404,01405,01406,01407,01408,01409,01423,01424,01425,01427,01428,01429,01430,01431,01432,01433,01434,01436,01437,01438,01452,01453,01454,01455'}, {'cdArea': '01456,01457,01458,01459,01460,01461,01462,01463,01464,01465,01468,01469,01470,01471,01472,01481,01482,01483,01484,01485,01486,01487,01511,01512,01513,01514,01516,01517,01518,01519,01520,01543,01544,01545,01546,01547,01549,01550,01552,01555,01559,01560,01561,01562,01563,01564,01571,01575,01578,01581,01584,01585,01586,01601,01602,01604,01607,01608,01609,01610,01631,01632,

## estat-APIのレスポンスをBigQueryに保存

DataFrameを整形する関数

In [112]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json
import pandas as pd

"""
estat-APIのレスポンスを整形してDataFrameに変換
"""

def get_estat_dataframe(params):
  res = get_estat_response(params)
  df_res = conv_estat_response_to_dataframe(res)
  df_res = format_estat_dataframe(df_res)

  return df_res

"""
estat-APIのレスポンスを取得する関数
"""

def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))


"""
estat-APIのレスポンスをDataFrameに変換する
"""

def conv_estat_response_to_dataframe(response):

  # VALUEをDataFrameに変換
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df = pd.merge(df, df_class, on=key_name, how='outer')

  # 統計情報を追加
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']
  df['statsDataId'] = TABLE_INF['@id']
  df['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df

"""
DataFrameを整形する
"""

def  format_estat_dataframe(df_arg):

  # 列を抽出して名前変更
  df =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','$','@unit']]
  df = df.rename(columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','$':'value','@unit':'unit'})

  # 欠損データ削除
  df = df.dropna(subset=['categoryCode'])

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df['timeCode'] = df['timeCode'].astype(str)
  df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
  
  # 地域情報をマージ
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    df_arealist = pd.json_normalize(arealist)  
  df = pd.merge(df, df_arealist, on='areaCode', how='right')


  df = df.dropna(subset=['categoryCode'])

  # '-'を'0'に置換
  df = df.replace({'value': {'-': '0'}})

  return df

In [ ]:
!pip install pandas-gbq

In [116]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# 統計カード一覧を取得
with open('/content/drive/MyDrive/statistics-hyogo/contents/statistics-cards.json') as j:
  cardlist = json.load(j)

for card in cardlist:

  # BigQueryのテーブル定義
  project_id='primal-buttress-342908'
  dataset_id = 'contents'
  table_id= card['cardId']

  client = bigquery.Client()

  try:
    client.get_table(f'{project_id}.{dataset_id}.{table_id}')  
    print("Table {} already exists.".format(table_id))

  except NotFound:
    estatParams = card['estatParams']

    df = get_estat_dataframe(estatParams)
    df.to_gbq(f'{dataset_id}.{table_id}', project_id=project_id, if_exists="replace")
    
    print(estatParams)
    print(f'{table_id}を保存しました')



Table total-population-prefecture already exists.
Table japanese-population-prefecture already exists.
Table population-pyramid-prefecture already exists.
Table median-age-prefecture already exists.
Table population-age-prefecture  already exists.
Table unmarried-prefecture already exists.
Table spouse-prefecture already exists.
Table bereavement-prefecture already exists.
Table separated-prefecture already exists.
Table unmarried-rate-pretecture already exists.
Table foreigner-population-prefecture already exists.
Table densely-populated-area-population-prefecture already exists.
Table densely-populated-area-prefecture already exists.
Table foreigner-prefecture already exists.
Table foreign-resident-prefecture already exists.
Table birth-prefecture already exists.
Table total-fertility-rate-prefecture already exists.
Table death-prefecture already exists.
Table death-pyramid-prefecture already exists.
Table age-adjusted-mortality-prefecture already exists.
Table stillbirth-prefecture 

1it [00:05,  5.56s/it]


{'statsDataId': '0000010101', 'cdCat01': 'A6106,A610601,A610602'}
inflow-outside-prefectureを保存しました


1it [00:05,  5.77s/it]


{'statsDataId': '0000010101', 'cdCat01': 'A6107,A610701,A610702,A6108,A6107,A6108'}
daytime-population-prefectureを保存しました


...

1it [00:18, 18.51s/it]


{'statsDataId': '0000020202', 'cdCat01': 'B1101,B1102'}
total-area-cityを保存しました


1it [00:11, 12.00s/it]


{'statsDataId': '0000020202', 'cdCat01': 'B1103'}
habitable-area-cityを保存しました


KeyboardInterrupt: ignored

# データ作成

BigQueryのテーブルをDataFrameに格納

In [83]:
import pandas as pd

def get_gbq(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'contents'  
  query = f'SELECT * FROM `{project_id}.{dataset_id}.{table_id}`'
  
  return pd.read_gbq(query, project_id, dialect='standard')

ランキングの付与

In [115]:
"""
都道府県のランキングを付与する関数
"""
def withrank_prefecture(df_arg):

  # 年度リスト、カテゴリリストの作成
  times = list(set(df_arg['timeCode'].tolist()))
  categories = list(set(df_arg['categoryCode'].tolist()))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  for time in times:
    for category in categories:
      df = df_arg.copy()
      df = df[df['timeCode'] == time][df['categoryCode'] == category]
      
      # valueを数値に変換して、'rank'を付与
      df = df.astype({'value': float})
      df['rankJapan'] = df.rank(ascending=False)['value'].astype(int).astype(str)
      
      # 結合
      df_res = pd.concat([df_res, df])
  
  return df_res

In [105]:
table_id = 'total-population-prefecture'
df = get_gbq(table_id)
df_rank = withrank_prefecture(df)
df_rank

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType,rank_japan
62,0000010101,社会・人口統計体系,A110101,総人口（男）,1992,1992年度,01000,2726000.0,人,0,1,北海道,None,None,None,北海道,prefecture,7
197,0000010101,社会・人口統計体系,A110101,総人口（男）,1992,1992年度,02000,701000.0,人,1,2,青森県,None,None,None,青森県,prefecture,28
332,0000010101,社会・人口統計体系,A110101,総人口（男）,1992,1992年度,03000,680000.0,人,2,3,岩手県,None,None,None,岩手県,prefecture,29
467,0000010101,社会・人口統計体系,A110101,総人口（男）,1992,1992年度,04000,1122000.0,人,3,4,宮城県,None,None,None,宮城県,prefecture,15
602,0000010101,社会・人口統計体系,A110101,総人口（男）,1992,1992年度,05000,581000.0,人,4,5,秋田県,None,None,None,秋田県,prefecture,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5684,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,43000,1841000.0,人,42,43,熊本県,None,None,None,熊本県,prefecture,22
5819,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,44000,1240000.0,人,43,44,大分県,None,None,None,大分県,prefecture,32
5954,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,45000,1172000.0,人,44,45,宮崎県,None,None,None,宮崎県,prefecture,36
6089,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,46000,1805000.0,人,45,46,鹿児島県,None,None,None,鹿児島県,prefecture,23


In [109]:
"""
市区町村のランキングを付与する関数
"""
def withrank_city(df_arg):

  # 年度リスト、カテゴリリストの作成
  times = list(set(df_arg['timeCode'].tolist()))
  categories = list(set(df_arg['categoryCode'].tolist()))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  for time in times:
    for category in categories:
      df = df_arg.copy()
      df = df[df['timeCode'] == time][df['categoryCode'] == category]
      
      # valueを数値に変換して、'rank'を付与
      df = df.astype({'value': float})
      df['rankJapan'] = df.rank(ascending=False)['value'].astype(int).astype(str)
      
      # 結合
      df_res = pd.concat([df_res, df])
  
  return df_res

In [111]:
table_id = 'total-population-city'
df = get_gbq(table_id)
df
# df_rank = withrank_city(df)
# df_rank

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,01100,1401757,人,0,1,北海道,01100,札幌市,2,札幌市,city
1,0000020201,社会・人口統計体系,A1101,総人口,1985,1985年度,01100,1542979,人,0,1,北海道,01100,札幌市,2,札幌市,city
2,0000020201,社会・人口統計体系,A1101,総人口,1990,1990年度,01100,1671742,人,0,1,北海道,01100,札幌市,2,札幌市,city
3,0000020201,社会・人口統計体系,A1101,総人口,1995,1995年度,01100,1757025,人,0,1,北海道,01100,札幌市,2,札幌市,city
4,0000020201,社会・人口統計体系,A1101,総人口,2000,2000年度,01100,1822368,人,0,1,北海道,01100,札幌市,2,札幌市,city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30628,0000020201,社会・人口統計体系,A110101,総人口（男）,2015,2015年度,47382,1147,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city
30629,0000020201,社会・人口統計体系,A110102,総人口（女）,2000,2000年度,47382,895,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city
30630,0000020201,社会・人口統計体系,A110102,総人口（女）,2005,2005年度,47382,857,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city
30631,0000020201,社会・人口統計体系,A110102,総人口（女）,2010,2010年度,47382,816,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city
